# Manipuler les prévision météorologiques avec Python

Debut 2024, Météo France à ouvert toutes ses données météorologiques au public, incluant les observations et les prévisions.

Ces données sont disponibles via [une API REST](https://portail-api.meteofrance.fr) ainsi que sous forme de fichier sur le site [meteo.data.gouv.fr](https://meteo.data.gouv.fr).

Dans ce tutoriel, nous allons voir comment manipuler les fichiers de prévision météo.

# Le format des fichiers

Les données météo sont généralement diffusées dans le format [GRIB](https://en.wikipedia.org/wiki/GRIB). C'est un format Binaire qui permet de stocker plusieurs champs de valeurs sur une même grille latitude/longitude/altitude/temps.

Chaque model météo (ARPEGE, AROME, ICON, etc.) ont plusieurs résolutions de grilles, de 0.1° à 0.025°.
Pour chaque résolution, les informations diffusées sont différentes.

Nous utiliserons ici les données de AROME à la résolution de 0.025°.

Meteo France diffuse les prévisions avec plusieurs fichiers afin de réduire la taille de chacun d'entre eux. La documentation exhaustive est [disponnible ici](https://www.data.gouv.fr/fr/datasets/r/3aa3ce62-1f69-4ea1-8157-a53eac61c6bb) mais en voici les principaux : 

- Champs de surfaces
    - **SP1** : inclue le vent a 10m d'haltitude (direction, vitesse, composantes $U$ et $V$), le flux solaire vertical, la précipitation, etc.
    - **SP2** : inclue la temperature au sol, la nebulositée, les valeurs min et max de la temperature à 2m
    - **SP3** : inclue le flux thermique vertical, le flux solair total
- champs isobares 
    - **IP1** à **IP5** avec les champs de temperature, humiditée, etc. sur les surfaces isobares de 100 à 1000hPa
- Champs de hauteurs
    - **HP1** a **HP3** avec la temperature, humiditée, vent, etc. sur 24 niveaux, de 20m à 3000m

Il faut donc, pour chaque champs désirer, choisir le bon fichier à récupérer et ouvire.

Dans se tutoriel, nous nous intéresserons à la vitesse du vent et au flux solaire vertical, donc nous utiliserons le fichier **SP1**.

# Récupérer le fichier

Les fichiers sont téléchargeables sur le site [meteo.data.gouv.fr](https://meteo.data.gouv.fr/datasets/65bd12d7bfd26e26804204cb). 
Il y a un fichier par horizon de prévision (de 1h à 42h) et par date à laquel la prévision a été faite.
Les fichiers sont donc nommés de la manière suivante :
```
arome__0025__HP1__00H06H__2024-05-21T03:00:00Z.grib2
~~~~~  ~~~~  ~~~  ~~~~~~  ~~~~~~~~~~~~~~~~~~~~~~~~~~~
  |     |     |     |                   |
  |     |     |     |                   +-- Date à laquel la prévision a été calculée
  |     |     |     +-- Horizon de la prévision
  |     |     +-- Type de champs
  |     +-- Résolution de la grille
  +-- Model
```
                    
A la date de rédaction de ce tutoriel, le 21 mai 2024, le fichier pour l'horizon 0h et la date 2024-05-21 03:00 est [disponible ici](https://object.data.gouv.fr/meteofrance-pnt/pnt/2024-05-21T03:00:00Z/arome/0025/SP1/arome__0025__SP1__00H06H__2024-05-21T03:00:00Z.grib2).

In [1]:
import requests
url = "https://object.data.gouv.fr/meteofrance-pnt/pnt/2024-05-21T03:00:00Z/arome/0025/SP1/arome__0025__SP1__00H06H__2024-05-21T03:00:00Z.grib2"
filename = "arome_sp1_oh.grib2"

In [2]:
r = requests.get(url, stream=True)
with open(filename, "wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

# Lire le fichier

Pour lire le fichier GRIB, nous utiliserons la librairie `cfgrib`.
Celle-ci est intégrée à `xarray` et permet de lire les fichiers GRIB directement.

Il vous faudra installer les modules
```
conda install -c conda-forge xarray cfgrib
```


In [2]:
import xarray as xr

ds = xr.open_dataset(filename, engine="cfgrib")

/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(
skipping variable: paramId==260065 shortName='gust'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 701, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 629, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='step' value=Variable(dimensions=('step',), data=array([0., 1., 2., 3., 4., 5., 6.])) new_value=Variable(dimensions=('step',), data=ar

DatasetBuildError: multiple values for unique key, try re-open the file with one of:
    filter_by_keys={'stepType': 'instant'}
    filter_by_keys={'stepType': 'max'}
    filter_by_keys={'stepType': 'accum'}

La fonction échoue. Comme l'indique le message d'erreur, c'est dû au fait que plusieurs variables sont fournit avec plusieurs type de valeure :
- la valeur instantané
- la valeur maximum
- la valeur accumulée

In [3]:
ds = xr.open_dataset(filename, engine="cfgrib", backend_kwargs={"filter_by_keys": {'stepType': 'instant'}})
ds

skipping variable: paramId==0 shortName='unknown'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 701, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 629, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='step' value=Variable(dimensions=('step',), data=array([0., 1., 2., 3., 4., 5., 6.])) new_value=Variable(dimensions=('step',), data=array([1., 2., 3., 4., 5., 6.]))
skipping variable: paramId==167 shortName='t2m'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib

<xarray.Dataset> Size: 113MB
Dimensions:            (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B ...
  * step               (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  float64 8B ...
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 56B ...
    meanSea            float64 8B ...
Data variables:
    wdir10             (step, latitude, longitude) float32 23MB ...
    si10               (step, latitude, longitude) float32 23MB ...
    u10                (step, latitude, longitude) float32 23MB ...
    v10                (step, latitude, longitude) float32 23MB ...
    prmsl              (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2024-05-21T12:01 GRIB to CDM+CF via cfgrib-0.9.1...

## Premières difficultées

Comme vous pouvez le voir lors de l'étapes précédente :
- un DataSet Xarray a bien été retournée
- ce DataSet ne présente que 5 variables concernant principalement les parameters du vent à 10m. Vous pouvez utiliser la sortie de la cellule précédentes pour inspecter les attributes et les méta-données des coordinée et des variables.
- plusieurs messages d'erreurs indiquent des incompatibilitées entre la valeure de l'altitude ainsi que pour les valeures.


## Solution 

La solution pour corriger ce problème est de filter les variables concaténé en DataSet. On pourra
1. Selectionner une variable directement
2. Filtrer sur le champ de l'altitude

## 1. Selection de la variable

Cette methode est la plus simple, mais necessite de connaitre le nom de la variable.

In [4]:
ds_u10 = xr.open_dataset(filename, engine="cfgrib", backend_kwargs={"filter_by_keys": {"shortName": "10u"}})
ds_u10

<xarray.Dataset> Size: 23MB
Dimensions:            (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B ...
  * step               (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  float64 8B ...
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 56B ...
Data variables:
    u10                (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2024-05-21T12:01 GRIB to CDM+CF via cfgrib-0.9.1...

## 2. Filtrer par altitude

Cette methode permet de selectionner les variables qui sont a la même altitude.

In [6]:
ds_surface = xr.open_dataset(filename, engine="cfgrib", backend_kwargs={"filter_by_keys": {'stepType': 'accum', "typeOfLevel": "surface"}})
ds_surface

<xarray.Dataset> Size: 77MB
Dimensions:     (step: 6, latitude: 717, longitude: 1121)
Coordinates:
    time        datetime64[ns] 8B ...
  * step        (step) timedelta64[ns] 48B 01:00:00 02:00:00 ... 06:00:00
    surface     float64 8B ...
  * latitude    (latitude) float64 6kB 55.4 55.38 55.35 ... 37.55 37.53 37.5
  * longitude   (longitude) float64 9kB -12.0 -11.97 -11.95 ... 15.95 15.98 16.0
    valid_time  (step) datetime64[ns] 48B ...
Data variables:
    tp          (step, latitude, longitude) float32 19MB ...
    sprate      (step, latitude, longitude) float32 19MB ...
    ssrd        (step, latitude, longitude) float32 19MB ...
    unknown     (step, latitude, longitude) float32 19MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2024-05-21T12:01 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
ds_2m = xr.open_dataset(filename, engine="cfgrib", backend_kwargs={"filter_by_keys": {"typeOfLevel": "heightAboveGround", "level": 2}})
ds_2m


<xarray.Dataset> Size: 45MB
Dimensions:            (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B ...
  * step               (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  float64 8B ...
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 56B ...
Data variables:
    t2m                (step, latitude, longitude) float32 23MB ...
    r2                 (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2024-05-21T12:01 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
ds_10m = xr.open_dataset(filename, engine="cfgrib", backend_kwargs={"read_keys": ["stepRange"],
                                                                    "filter_by_keys": {
                                                                                      "typeOfLevel": "heightAboveGround",
                                                                                      "level": 10,
                                                                                       }
                                                                    })
ds_10m

skipping variable: paramId==260065 shortName='gust'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 701, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 629, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='step' value=Variable(dimensions=('step',), data=array([0., 1., 2., 3., 4., 5., 6.])) new_value=Variable(dimensions=('step',), data=array([1., 2., 3., 4., 5., 6.]))


skipping variable: paramId==260646 shortName='efg10'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 701, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-packages/cfgrib/dataset.py", line 629, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='step' value=Variable(dimensions=('step',), data=array([0., 1., 2., 3., 4., 5., 6.])) new_value=Variable(dimensions=('step',), data=array([1., 2., 3., 4., 5., 6.]))
skipping variable: paramId==260647 shortName='nfg10'
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/hatch/env/virtual/antoinetavant-github-io/vsXjhu16/antoinetavant-github-io/lib/python3.10/site-package

<xarray.Dataset> Size: 90MB
Dimensions:            (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B ...
  * step               (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  float64 8B ...
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 56B ...
Data variables:
    wdir10             (step, latitude, longitude) float32 23MB ...
    si10               (step, latitude, longitude) float32 23MB ...
    u10                (step, latitude, longitude) float32 23MB ...
    v10                (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2024-05-21T12:01 GRIB to CDM+CF via cfgrib-0.9.1...

Pour les variables à 10 m d'altitude, nous avons un nouvel message d'erreur.
C'est du au fait que certaines variables sont fournies avec un horizon de 0h (ce qui correspond a la valeur initiale donc) et d'autres uniquement à partir de 1h d'horizon.

Actuellement, je n'ai pas réussi à filtrer sur ce parametre. 
Une solution est de laisser `cfgrib` determiner lui-même les differents groupes

In [9]:
import cfgrib
list_dataset = cfgrib.open_datasets(filename)
print(f"number of datasets: {len(list_dataset)}")

number of datasets: 6


In [10]:
for ds in list_dataset:
    display(ds)

<xarray.Dataset> Size: 90MB
Dimensions:            (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B 2024-05-21T03:00:00
  * step               (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 56B 2024-05-21T03:00:00 ... 2024...
Data variables:
    u10                (step, latitude, longitude) float32 23MB ...
    v10                (step, latitude, longitude) float32 23MB ...
    si10               (step, latitude, longitude) float32 23MB ...
    wdir10             (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse

<xarray.Dataset> Size: 45MB
Dimensions:            (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B 2024-05-21T03:00:00
  * step               (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  float64 8B 2.0
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 56B 2024-05-21T03:00:00 ... 2024...
Data variables:
    t2m                (step, latitude, longitude) float32 23MB ...
    r2                 (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse

<xarray.Dataset> Size: 58MB
Dimensions:            (step: 6, latitude: 717, longitude: 1121)
Coordinates:
    time               datetime64[ns] 8B 2024-05-21T03:00:00
  * step               (step) timedelta64[ns] 48B 01:00:00 02:00:00 ... 06:00:00
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 6kB 55.4 55.38 55.35 ... 37.53 37.5
  * longitude          (longitude) float64 9kB -12.0 -11.97 ... 15.98 16.0
    valid_time         (step) datetime64[ns] 48B 2024-05-21T04:00:00 ... 2024...
Data variables:
    gust               (step, latitude, longitude) float32 19MB ...
    efg10              (step, latitude, longitude) float32 19MB ...
    nfg10              (step, latitude, longitude) float32 19MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse

<xarray.Dataset> Size: 23MB
Dimensions:     (step: 7, latitude: 717, longitude: 1121)
Coordinates:
    time        datetime64[ns] 8B 2024-05-21T03:00:00
  * step        (step) timedelta64[ns] 56B 00:00:00 01:00:00 ... 06:00:00
    meanSea     float64 8B 0.0
  * latitude    (latitude) float64 6kB 55.4 55.38 55.35 ... 37.55 37.53 37.5
  * longitude   (longitude) float64 9kB -12.0 -11.97 -11.95 ... 15.95 15.98 16.0
    valid_time  (step) datetime64[ns] 56B ...
Data variables:
    prmsl       (step, latitude, longitude) float32 23MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse

<xarray.Dataset> Size: 77MB
Dimensions:     (step: 6, latitude: 717, longitude: 1121)
Coordinates:
    time        datetime64[ns] 8B 2024-05-21T03:00:00
  * step        (step) timedelta64[ns] 48B 01:00:00 02:00:00 ... 06:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 6kB 55.4 55.38 55.35 ... 37.55 37.53 37.5
  * longitude   (longitude) float64 9kB -12.0 -11.97 -11.95 ... 15.95 15.98 16.0
    valid_time  (step) datetime64[ns] 48B 2024-05-21T04:00:00 ... 2024-05-21T...
Data variables:
    unknown     (step, latitude, longitude) float32 19MB nan nan nan ... nan nan
    ssrd        (step, latitude, longitude) float32 19MB ...
    tp          (step, latitude, longitude) float32 19MB ...
    sprate      (step, latitude, longitude) float32 19MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse

<xarray.Dataset> Size: 19MB
Dimensions:     (step: 6, latitude: 717, longitude: 1121)
Coordinates:
    time        datetime64[ns] 8B 2024-05-21T03:00:00
  * step        (step) timedelta64[ns] 48B 01:00:00 02:00:00 ... 06:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 6kB 55.4 55.38 55.35 ... 37.55 37.53 37.5
  * longitude   (longitude) float64 9kB -12.0 -11.97 -11.95 ... 15.95 15.98 16.0
    valid_time  (step) datetime64[ns] 48B 2024-05-21T04:00:00 ... 2024-05-21T...
Data variables:
    unknown     (step, latitude, longitude) float32 19MB nan nan nan ... nan nan
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse

On retrouve alors les differents champs indiquées dans la documentations :
1. Le premier dataset correspond aux variables à 10 m d'altitude avec 7 pas de temps, donc les parametres de vent : `u10`, `v10`, `wdir10` et `si10`
2. le deuxième dataset correspond aux variables à 2 m d'altitude avec 7 pas de temps, donc les parametres de temperature et humidité: `t2m`, `r2m`
3. le troisième dataset correspond aux variables à 10 m d'altitude avec 6 pas de temps, donc les parametres de rafales de vent: `gust`, `efg10`, `nfg10`
4. le quatrième dataset correspond aux variables à la surface de la mer avec 7 pas de temps, donc les parametres de pression au niveau de la mer: `prmsl`
5. le cinquième dataset correspond aux variables à la surface du sol avec 7 pas de temps, donc les parametres de radiation solaire et de précipitation: `ssrd`, `sprate`, `tp`, et un `Unknown` qui n'est pas documenté
6. le sixième dataset correspond aux variables à la surface du sol avec 6 pas de temps, mais il n'est pas documenté

A partir de la documentation, on peut en deduire que les 2 variables non documentées sont:
- Graupel (précipitation dite de neige roulée)
- Nebulosity (nuage)

# Conclusion

Nous avons vu comment lire un fichier de prévision météo fournies par Météo France.

Nous avons vu comment filtrer les variables pour ne garder que celles qui nous intéressent, en utilisant les paramètres de l'altitude.

Nous avons finalement vu comment laisser `cfgrib` déterminer lui-même les groupes de variables.

Les fichiers GRIB fournis par Météo France ne sont pas faciles à manipuler, en particulier à cause de la variété des indexes et des horizons de prévision toutes incluses dans un seul fichier.
Nous verrons dans un prochain tutoriel comment utiliser l'API REST pour récupérer directement les variables qui nous intéressent.